In [1]:
import substrateinterface
from substrateinterface import SubstrateInterface, Keypair
from substrateinterface.exceptions import SubstrateRequestException

substrate = SubstrateInterface(
    url="https://127.0.0.1:9944",
    ss58_format=42,
    type_registry_preset='kusama'
)

In [2]:
# Function to make extrinsic calls
def make_call(call_module, call_function, call_params, keypair, wait_for_inclusion = True, wait_for_finalization = False):
    call = substrate.compose_call(
        call_module=call_module,
        call_function=call_function,
        call_params=call_params
    )

    extrinsic = substrate.create_signed_extrinsic(call=call, keypair=keypair)

    try:
        receipt = substrate.submit_extrinsic(extrinsic, wait_for_inclusion=wait_for_inclusion, wait_for_finalization=wait_for_finalization)
        print("Extrinsic '{}' sent and included in block '{}'".format(receipt.extrinsic_hash, receipt.block_hash))

    except SubstrateRequestException as e:
        print("Failed to send: {}".format(e))
    return receipt

In [3]:
alice = Keypair.create_from_uri('//Alice')
bob = Keypair.create_from_uri('//Bob')

In [4]:
make_call("Msa", "create", {}, alice)
make_call("Msa", "create", {}, bob)

SSLError: HTTPSConnectionPool(host='127.0.0.1', port=9944): Max retries exceeded with url: / (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:1131)')))

# Create MSA for brand new account
Use this when user wants control of thier keys and will pay for their own transactions

In [ ]:
# Create never before used wallet
new_wallet = Keypair.create_from_uri('//ChangeThisNamefgnf', crypto_type=substrateinterface.KeypairType.SR25519)

In [ ]:
# Give wallet some tokens to it can make MSA
# Must be at least 1 unit
one_unit = 1000000000000
receipt = make_call("Balances", "transfer", {"dest": new_wallet.ss58_address, "value": 10 * one_unit}, alice)
receipt.error_message

In [ ]:
def get_msa_id(wallet):
    msa_key = substrate.query(
        module='Msa',
        storage_function='KeyInfoOf',
        params=[wallet.ss58_address]
    )

    if msa_key == None:
        make_call("Msa", "create", {}, wallet)
        msa_key = substrate.query(
            module='Msa',
            storage_function='KeyInfoOf',
            params=[wallet.ss58_address]
        )

    msa_id = msa_key['msa_id'].decode()
    return msa_id

In [ ]:
msa_id = get_msa_id(new_wallet)

## Delegate MSA to a provider

In [ ]:
payload_raw = { "authorized_msa_id": msa_id, "permission": 0 }

In [ ]:
def get_signature(payload, signer):
    # encode payload using SCALE
    # I found scale_info from "substrate.metadata_decoder"
    payload_encoded = substrate.encode_scale(type_string='scale_info::8', value=payload['authorized_msa_id']) + \
                            substrate.encode_scale(type_string='scale_info::2', value=payload['permission'])

    # Payload must be wrapped in theses Bytes objects
    payload_encoded = "<Bytes>".encode() + payload_encoded.data + "</Bytes>".encode()

    # The provider address signs the payload, so in this case alice
    return signer.sign(payload_encoded)

In [ ]:
signature = get_signature(payload_raw, alice)

In [ ]:
# parameters for call function
call_params = {
    "provider_key": alice.ss58_address,
    "proof": {"Sr25519": "0x" + signature.hex()},
    "add_provider_payload": payload_raw
}

receipt = make_call("Msa", "add_provider_to_msa", call_params, new_wallet)
print(receipt.error_message)

# Following code is run instead of above if user doesn't want control of keys and allows provider to pay for transactions

In [ ]:
# Create never before used wallet
new_wallet = Keypair.create_from_uri('//sdvsbsdfghdfdvsvs', crypto_type=substrateinterface.KeypairType.SR25519)

In [ ]:
# get provider msa id, so alice in this case
provider_msa_id = get_msa_id(alice)

In [ ]:
payload_raw = { "authorized_msa_id": provider_msa_id, "permission": 0 }

# This time new_wallet signs payload and therefore doesn't need to pay any gas
signature = get_signature(payload_raw, new_wallet)

In [ ]:
call_params = {
    "delegator_key": new_wallet.ss58_address,
    "proof": {"Sr25519": "0x" + signature.hex()},
    "add_provider_payload": payload_raw
}

# provider signs this
receipt = make_call("Msa", "create_sponsored_account_with_delegation", call_params, alice)
print(receipt.error_message)

In [ ]:
wallets_msa_id = get_msa_id(new_wallet)

# Create Schema and add Messages

In [ ]:
# define schema and then check if it exists already, if not, then mint it
# We will probably turn schema check into api call to save time

schema = "subreddit,author,title,selftext,url,is_nsfw,dsvs"

schema_count = substrate.query(
    module='Schemas',
    storage_function='SchemaCount',
    params=[]
).value

schemaId = -1
for i in range(1, schema_count+1):
    schemaTemp = substrate.query(
        module='Schemas',
        storage_function='Schemas',
        params=[i]
    )
    if schemaTemp == schema:
        schemaId = i
        print(schemaTemp.value)
        break
    
if schemaId == -1:
    receipt = make_call("Schemas", "register_schema", {"schema": schema}, alice)
    for event in receipt.triggered_events:
        event = event.decode()
        if event['event']['event_id'] == 'SchemaRegistered':
            schemaId = event['event']['attributes'][1]

print(schemaId)

In [ ]:
substrate.query(
    module='Balances',
    storage_function='Account',
    params=[new_wallet.ss58_address]
)

In [ ]:
message = "AskReddit,LazarShockX,What are some good ice breaker questions?,,https://www.reddit.com/r/AskReddit/comments/v5ihpk/what_are_some_good_ice_breaker_questions/,False"

call_params = {
    "on_behalf_of": wallets_msa_id,
    "schema_id": schemaId,
    "message": message
}
receipt = make_call("Messages", "add", call_params, alice, wait_for_inclusion=True)
wallets_msa_id

# result = substrate.subscribe_block_headers(subscription_handler)

In [ ]:
receipt.triggered_events

# Query data
The following use substrate's rpc_request function to call rpc methods that are implemented for the pallets
This doesn't seem to be implemented by the polkadot or substrate frontends so some code might need to be changed

In [ ]:
substrate.get_block()['header']['number']

In [ ]:
params = [
    schemaId,
    {
        "page_size": 10000,
        "from_block": 0,
        "to_block": 10000,
        "from_index": 1,
    }
]

data = substrate.rpc_request(
    method='messages_getBySchema',
    params=params,
)
data

In [ ]:
# data can be converted back to original string
bytes.fromhex(data['result']['content'][0]['data'][2:]).decode()

In [ ]:
provider_msa_id

In [ ]:
alice.ss58_address

In [ ]:
params = [1]

substrate.rpc_request(
    method='msa_getMsaKeys',
    params=params,
)

In [ ]:
params = [alice.ss58_address]

substrate.rpc_request(
    method='msa_getMsaId',
    params=params,
)

In [ ]:
params = [list(range(12)), 3]

substrate.rpc_request(
    method='msa_checkDelegations',
    params=params,
)

In [ ]:
params = [2]

substrate.rpc_request(
    method='schemas_getBySchemaId',
    params=params,
)

In [ ]:
# params = [_,'subreddit,author,title,selftext,url,is_nsfw,dsvs'.encode().hex()]

# substrate.rpc_request(
#     method='schemas_checkSchemaValidity',
#     params=params,
# )

# Below is test code

In [ ]:
def subscription_handler(obj, update_nr, subscription_id):

    print(f"New block #{obj['header']['number']} produced b")
    print(obj)

    if update_nr > 10:
        return {'message': 'Subscription will cancel when a value is returned', 'updates_processed': update_nr}


# result = substrate.subscribe_block_headers?

In [ ]:
t = substrate.get_block(substrate.get_block_hash(2662))

In [ ]:
t = substrate.get_runtime_events(substrate.get_block_hash(2662))

In [ ]:
t['result'][0]

In [ ]:
t['result']

In [ ]:
i = 0
block_hash = substrate.get_block_hash(i)
while (substrate.block_hash != block_hash):
    for event in substrate.get_runtime_events(block_hash)['results']:
        if event['event_id'] == 'MessagesStored':
            
    block_hash = substrate.get_block_hash(i)

In [ ]:
substrate.query_map('System', 'Events', page_size=200, max_results=400)

In [ ]:
substrate.query(
    module='System',
    storage_function='Events',
    params=[],
    block_hash=substrate.get_block_hash(2)
)

In [ ]:
substrate.get_block("0x9dbe5e6b82ac046ddd30b1a3c99160f53a8380ca4b461c93b9335273d8b0a066")

In [ ]:
substrateinterface.utils.hasher.xxh128("Sudo".encode()) + substrateinterface.utils.hasher.xxh128("Key".encode())

In [ ]:
substrate.rpc_request?

In [ ]:
provider_msa_id

In [ ]:
schemaId

In [ ]:
params = [
    2,
    {
        "page_size": 10000,
        "from_block": 0,
        "to_block": 10000,
        "from_index": 1,
    }
#     substrateinterface.utils.hasher.xxh128("Messages".encode()) + substrateinterface.utils.hasher.xxh128("BlockMessages".encode()),
#     substrate.get_block_hash(2)
]

substrate.rpc_request(
    method='messages_getBySchema',
    params=params,
)

In [ ]:
s

In [ ]:
t = data['result']['content'][0]['data']

In [ ]:
bytes.fromhex(data['result']['content'][0]['data'][2:]).decode()

In [ ]:
bytes.fromhex(message.encode().hex()).decode()

In [ ]:
message.encode().hex()

In [ ]:
substrate.chain